<a href="https://colab.research.google.com/github/saadali112/CS-351L---AI-Lab-GitHub-Repository_2022512/blob/main/2022512_CS351L_Lab02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import heapq  # Importing heapq to implement the priority queue
import random

# Function to create a grid based on user-defined size and randomly place Pac-Man and ghosts
def create_pacman_grid(size):
    grid = [[' ' for _ in range(size)] for _ in range(size)]
    pacman_x, pacman_y = 0, 0
    grid[pacman_x][pacman_y] = 'P'  # Place Pac-Man at the top-left corner

    # Place ghosts ('G') at random positions, ensuring they're not on Pac-Man's start
    ghosts = []
    num_ghosts = 3  # Let's place 3 ghosts for simplicity
    for _ in range(num_ghosts):
        ghost_x, ghost_y = random.randint(0, size-1), random.randint(0, size-1)
        while (ghost_x, ghost_y) == (pacman_x, pacman_y):  # Ensure ghosts don't overlap with Pac-Man
            ghost_x, ghost_y = random.randint(0, size-1), random.randint(0, size-1)
        ghosts.append((ghost_x, ghost_y))
        grid[ghost_x][ghost_y] = 'G'  # Place a ghost

    return grid, (pacman_x, pacman_y), ghosts  # Return the grid, Pac-Man's position, and ghosts' positions

# Function to add random obstacles to the grid
def add_obstacles(grid, num_obstacles):
    size = len(grid)
    for _ in range(num_obstacles):
        obstacle_x, obstacle_y = random.randint(0, size-1), random.randint(0, size-1)
        while grid[obstacle_x][obstacle_y] in ['P', 'G']:  # Don't overwrite Pac-Man or ghosts
            obstacle_x, obstacle_y = random.randint(0, size-1), random.randint(0, size-1)
        grid[obstacle_x][obstacle_y] = 'X'  # Place an obstacle
    return grid

# Heuristic function for A* (using Manhattan distance)
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

# A* Search function for pathfinding
def a_star(grid, start, goal):
    size = len(grid)
    open_set = []  # Priority queue (min-heap)
    heapq.heappush(open_set, (0, start))  # Start with the initial position with a cost of 0
    came_from = {}  # Dictionary to store the parent of each state for path reconstruction
    g_score = {start: 0}  # Distance from start to each state
    f_score = {start: heuristic(start, goal)}  # Total cost estimate for each state

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Possible directions (up, down, left, right)

    while open_set:
        _, current = heapq.heappop(open_set)  # Get the position with the lowest cost

        # If the current position reaches the goal, stop the search
        if current == goal:
            break

        # Explore neighboring positions
        for direction in directions:
            next_x, next_y = current[0] + direction[0], current[1] + direction[1]
            next_state = (next_x, next_y)

            # Check if the next position is valid
            if 0 <= next_x < size and 0 <= next_y < size and grid[next_x][next_y] != 'X':
                # Calculate tentative g_score
                tentative_g_score = g_score[current] + 1

                # If this path to the neighbor is shorter, or it's the first time visiting
                if next_state not in g_score or tentative_g_score < g_score[next_state]:
                    came_from[next_state] = current
                    g_score[next_state] = tentative_g_score
                    f_score[next_state] = tentative_g_score + heuristic(next_state, goal)
                    heapq.heappush(open_set, (f_score[next_state], next_state))  # Add to the open set

    # Reconstruct the path
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = came_from.get(current, start)
    path.append(start)
    path.reverse()
    return path

# Function to print the grid with lines and the path marked
def print_grid_with_path(grid, path, pacman_position, goal):
    grid_with_path = [row.copy() for row in grid]
    for (x, y) in path:
        if grid_with_path[x][y] not in ['P', 'G']:  # Don't overwrite Pac-Man or ghosts
            grid_with_path[x][y] = '*'  # Mark the path

    # Place Pac-Man and the destination on the grid
    pacman_x, pacman_y = pacman_position
    goal_x, goal_y = goal
    grid_with_path[pacman_x][pacman_y] = 'P'
    grid_with_path[goal_x][goal_y] = 'F'  # Mark the finish point

    # Print the grid with lines separating cells
    print("\nGrid with Path:")
    print('-' * (len(grid_with_path) * 4 + 1))
    for row in grid_with_path:
        print('| ' + ' | '.join(row) + ' |')
        print('-' * (len(grid_with_path) * 4 + 1))

# Main function to simulate the Pac-Man game
def pacman_game():
    size = int(input("Enter the grid size (e.g., 6 for a 6x6 grid): "))
    num_obstacles = int(input(f"Enter the number of obstacles (less than {size * size - 4}): "))  # Adjust based on grid size

    # Create the grid and place Pac-Man and ghosts
    grid, pacman_position, ghosts = create_pacman_grid(size)

    # Set the goal (finish point) for Pac-Man at the bottom-right corner
    goal = (size - 1, size - 1)

    # Add random obstacles
    grid = add_obstacles(grid, num_obstacles)

    # Print the initial grid
    print("\nInitial Grid:")
    print_grid_with_path(grid, [], pacman_position, goal)  # No path yet

    # Calculate the A* path for Pac-Man to reach the goal
    print("\nPac-Man is trying to reach the finish point...")
    pacman_path = a_star(grid, pacman_position, goal)
    print_grid_with_path(grid, pacman_path, pacman_position, goal)  # Show Pac-Man's path

    # For each ghost, calculate the A* path to Pac-Man's starting position and print the grid with the path
    for ghost in ghosts:
        print(f"\nGhost at {ghost} is chasing Pac-Man...")
        ghost_path = a_star(grid, ghost, pacman_position)  # Find the path from ghost to Pac-Man
        print_grid_with_path(grid, ghost_path, pacman_position, goal)  # Show the path

# Run the Pac-Man game
pacman_game()


Enter the grid size (e.g., 6 for a 6x6 grid): 5
Enter the number of obstacles (less than 21): 15

Initial Grid:

Grid with Path:
---------------------
| P |   | X | X |   |
---------------------
| X |   | X | X | X |
---------------------
|   | X | X |   | G |
---------------------
|   |   | G |   | X |
---------------------
| X |   | X | G | F |
---------------------

Pac-Man is trying to reach the finish point...

Grid with Path:
---------------------
| P |   | X | X |   |
---------------------
| X |   | X | X | X |
---------------------
|   | X | X |   | G |
---------------------
|   |   | G |   | X |
---------------------
| X |   | X | G | F |
---------------------

Ghost at (2, 4) is chasing Pac-Man...

Grid with Path:
---------------------
| P |   | X | X |   |
---------------------
| X |   | X | X | X |
---------------------
|   | X | X |   | G |
---------------------
|   |   | G |   | X |
---------------------
| X |   | X | G | F |
---------------------

Ghost at (4, 3) is chas